In [1]:
# ## Download packages
# !pip install keras-vggface
# !pip install opencv-python
# !pip install tensorflow
# !pip install shutil
# !pip install matplotlib
# !pip install sklearn
# !pip install keras-applications
# !pip install tensorflow-gpu

In [1]:
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics

from keras_vggface.vggface import VGGFace
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout
from keras.callbacks import ReduceLROnPlateau

from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [2]:
import os

base_path = os.getcwd()

train_location = os.path.join(base_path, 'train')
val_location = os.path.join(base_path, 'val')

train_fake_image_location = os.path.join(train_location, 'fake_image')
train_real_image_location = os.path.join(train_location, 'real_image')
val_fake_image_location = os.path.join(val_location, 'fake_image')
val_real_image_location = os.path.join(val_location, 'real_image')

SHAPE=224

In [3]:
import shutil
import os
def squash_and_merge(directory):
    count = 0
    folders = os.listdir(directory)
    for f in folders :
        files = os.listdir(os.path.join(directory, f))
        for f_x in files :
            original = os.path.join(directory, f, f_x)
            target = os.path.join(directory, f'{count}.jpg')
            count += 1
            shutil.move(original, target)
        shutil.rmtree(os.path.join(directory, f))

In [4]:
# squash_and_merge(train_fake_image_location)
# squash_and_merge(train_real_image_location)
# squash_and_merge(val_fake_image_location)
# squash_and_merge(val_real_image_location)

In [5]:
train_new_location = os.path.join(base_path, 'train_1')
val_new_location = os.path.join(base_path, 'val_1')
try :
    shutil.copytree(train_fake_image_location,os.path.join(train_new_location, 'fake'))
    shutil.copytree(train_real_image_location,os.path.join(train_new_location, 'real'))
    shutil.copytree(val_fake_image_location,os.path.join(val_new_location, 'fake'))
    shutil.copytree(val_real_image_location,os.path.join(val_new_location, 'real'))
except FileExistsError:
    pass
finally :
    print('Copied')

Copied


In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255.)

batch_size = 32

train_flow = train_datagen.flow_from_directory(
    train_new_location,
    target_size=(SHAPE, SHAPE),
    batch_size=batch_size,
    class_mode='binary',
)

valid_flow = val_datagen.flow_from_directory(
    val_new_location,
    target_size=(SHAPE, SHAPE),
    batch_size=batch_size,
    class_mode='binary',
)



Found 66722 images belonging to 2 classes.
Found 12592 images belonging to 2 classes.


In [9]:
train_flow.class_indices

{'fake': 0, 'real': 1}

In [10]:
valid_flow.class_indices

{'fake': 0, 'real': 1}

In [11]:
"""
Plot the training and validation loss
epochs - list of epoch numbers
loss - training loss for each epoch
val_loss - validation loss for each epoch
"""
def plot_loss(epochs, loss, val_loss):
    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
    
    
"""
Plot the training and validation accuracy
epochs - list of epoch numbers
acc - training accuracy for each epoch
val_acc - validation accuracy for each epoch
"""
def plot_accuracy(epochs, acc, val_acc):
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label = 'Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [13]:
from tensorflow import keras

vgg_model = VGGFace(include_top=False, input_shape = (SHAPE,SHAPE,3))

last_layer = vgg_model.get_layer('pool5').output
flat_layer = Flatten(name='flatten')(last_layer)
fc1 = Dense(2048, activation='relu', name='fc1')(flat_layer)
dense2 = Dense(1, activation='sigmoid', name='dense2')(fc1)

custom_vgg_model = Model(vgg_model.input, dense2)


In [14]:
custom_vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0   

In [16]:
custom_vgg_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0001),
     metrics=['accuracy']
)

In [18]:

from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
checkpoint = ModelCheckpoint(filepath='customVGG.v2.1.h5',
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             monitor='val_loss'
                            )
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=3, 
                              verbose=1, 
                              min_delta=0.0001
                             )
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [19]:
train_steps = 66722//batch_size
valid_steps = 12592//batch_size

history = custom_vgg_model.fit_generator(
    train_flow,
    epochs=5,
    callbacks=callbacks,
    steps_per_epoch=train_steps,
    validation_data=valid_flow,
    validation_steps=valid_steps
)


d:\coding\python\misc\venv\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
2085/2085 [==============================] - 469s 217ms/step - loss: 0.4511 - accuracy: 0.8160 - val_loss: 0.3924 - val_accuracy: 0.8309

Epoch 00001: val_loss improved from inf to 0.39239, saving model to customVGG.v2.1.h5
Epoch 2/5
2085/2085 [==============================] - 436s 209ms/step - loss: 0.3619 - accuracy: 0.8432 - val_loss: 0.3653 - val_accuracy: 0.8405

Epoch 00002: val_loss improved from 0.39239 to 0.36535, saving model to customVGG.v2.1.h5
Epoch 3/5
 517/2085 [======>.......................] - ETA: 5:09 - loss: 0.3213 - accuracy: 0.8631

KeyboardInterrupt: 

In [15]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

KeyError: 'acc'

In [ ]:
plot_loss(range(1, len(loss) + 1), loss, val_loss)

In [ ]:
plot_accuracy(range(1, len(loss) + 1), acc, val_acc)

## Evaluation 


In [20]:
from tensorflow import keras
import pandas as pd
model = keras.models.load_model('customVGG.v2.1.h5')
test_location = os.path.join(base_path, 'test')

df = pd.read_csv(os.path.join(test_location,'image_predictions.csv'))
df

,folder,prediction
0,image/00000,NaN
1,image/00001,NaN
2,image/00002,NaN
3,image/00003,NaN
4,image/00004,NaN
...,...,...
785,image/00785,NaN
786,image/00786,NaN
787,image/00787,NaN
788,image/00788,NaN


In [21]:
import shutil
def predict(row, base):
    val = row['folder'].split('/')
    test_location = os.path.join(base, val[0], val[1])
    files = os.listdir(test_location)
    
    try:
        os.mkdir(os.path.join(test_location, 'hello'))
    except FileExistsError:
        pass
    for file in files:
        if (file.endswith('.jpg')):
            try :
                from_location = os.path.join(test_location, file)
                to_location = os.path.join(test_location,'hello', file)
                shutil.copy(from_location, to_location)
            except FileExistsError:
                pass
    new_test_location = os.path.join(test_location)
    
    test_flow = val_datagen.flow_from_directory(
        test_location,
        target_size=(SHAPE, SHAPE),
        batch_size=1,
        shuffle=False,
    )
    res = model.predict(test_flow)
    return np.mean(res)
    

In [22]:
df['prediction'] = df.apply(lambda row : predict(row, test_location), axis=1)

Found 16 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 21 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Found 72 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 18 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 25 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 16 images belonging to 1 classes.


Found 19 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 23 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 18 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 25 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 12 images belonging to 1 classes.


Found 19 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 21 images belonging to 1 classes.
Found 18 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 45 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 10 images belonging to 1 classes.


Found 14 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 32 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 18 images belonging to 1 classes.
Found 34 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 18 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 19 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
Found 29 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 14 images belonging to 1 classes.
Found 17 images belonging to 1 classes.
F

In [23]:
df

,folder,prediction
0,image/00000,0.000725
1,image/00001,0.102251
2,image/00002,0.034695
3,image/00003,0.081306
4,image/00004,0.062436
...,...,...
785,image/00785,0.228416
786,image/00786,0.218887
787,image/00787,0.109320
788,image/00788,0.107137


In [24]:
df.to_csv('simyujie_e0310402@u.nus.edu.csv', index=False)

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()